EBU6505 Reasoning and Agents

Introduction to Test-Time Compute
===

**Dr Chao Shu (chao.shu@qmul.ac.uk)**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Optional, Callable
import time
import re
from tqdm.notebook import tqdm

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown, display

## Introduction
---

While train-time compute scaling has dominated LLM progress in recent years, test-time compute scaling is emerging as a valuable complementary approach.
- Traditional scaling requires enormous pretraining budgets and resources, with costs approaching billions of dollars for the largest training clusters.
- Test-time compute scaling offers a more sustainable alternative by:
  - Using dynamic inference strategies
  - Allowing models to "think longer" on harder problems
  - Allocating computational resources based on task difficulty
- According to Ilya's speculation in his talk ["Sequence2Sequence Learning with Neural Networks: What a Decade"](https://www.youtube.com/embed/1yvBqasHLZs) at NeurIPS 2024, "Pre-Training as we know it will end" if there are no more new data. Inference/test time compute can be a strategy to enhance LLM reasoning capabilities without changing the model weights.
 - OpenAI o1/o3 model, which shows consistent improvement on difficult math problems as test-time compute increases.

![OpenAI o1 train-time and test-time compute](./imgs/L03_OpenAI_o1_compute-dark.webp)

Source: [OpenAI, Learning to Reason with LLMs](https://openai.com/index/learning-to-reason-with-llms/)

While Chain of Thought (CoT) provides a foundational reasoning approach, we can extend and improve it with more sophisticated search and sampling techniques.

In this lesson, we'll explore several test-time compute algorithms that build upon CoT reasoning. These approaches leverages additional compute at inference/test time rather than tuning the model weights.

## Tree of Thoughts (ToT)
---

Tree of Thought (ToT) extends Chain of Thought by exploring multiple reasoning reasoning paths over thoughts, creating a tree-like structure of possible reasoning paths [(Yao et al., 2023)](https://arxiv.org/abs/2305.10601).

![From CoT to ToT](./imgs/L03_ToT_Diagram.png)

Image Source: [Yao et al., Tree of Thoughts: Deliberate Problem Solving with Large Language Models](https://arxiv.org/abs/2305.10601)

**Algorithm Components**:

1. **Thought decomposition**: Break problem solving into smaller thought units.
2. **Thought generator**: At each step, generate k different thoughts (branches).
3. **State Evaluator**: Use **self-evaluation** or external evaluation to identify which branches to pursue.
4. **Search algorithms**: Exploration from the most promising nodes based on BFS or DFS.

![ToT BFS Example](./imgs/L03_ToT_Example.png)
Image Source: [Yao et al., Tree of Thoughts: Deliberate Problem Solving with Large Language Models](https://arxiv.org/abs/2305.10601)

**Example**:
- Decompose the thoughts into 3 steps, each an intermediate equation.
- At each tree node, exact the remaining numbers and prompt the LM to propose some possible next steps. 
- The same “propose prompt” is used for all 3 thought steps, though it only has one example with 4 input numbers. 
- Perform a breadth-first search (BFS) in ToT, where at each step we keep the best b = 5 candidates. Prompt LM to evaluate each thought candidate as “sure/maybe/impossible” with regard to reaching 24. The aim is to promote correct partial solutions that can be verdicted within few lookahead trials, and eliminate impossible partial solutions based on “too big/small” commonsense, and keep the rest “maybe”. We sample values 3 times for each thought.


**Key Characteristics**:
- Systematic exploration of the solution space
- Combines deliberate search with neural generation
- Can handle problems requiring planning and lookahead

**Limitation**
- Rely heavily on prompt engineering, difficult to generalise for different tasks.
- Rely on self-refinement capability of the model itself (the official repo use GPT-4o).

### Implementation

- [Yao et al. (2023)](https://arxiv.org/abs/2305.10601) open-sourced their implementation in the repo [here](https://github.com/princeton-nlp/tree-of-thought-llm/tree/master). You can play with it if interested.
- [Hulbert (2023)](https://github.com/dave1010/tree-of-thought-prompting) proposed Tree-of-Thought prompting that employs key elements from ToT frameworks as a straightforward prompting method without codes, allowing the LLM to assess intermediate thoughts within a single prompt. A sample ToT prompt is:

```text
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...
```

## Best-of-N Sampling
---

Language models often produce varied outputs for the same prompt due to their probabilistic nature. The quality of these outputs can vary significantly, especially for complex tasks.

**Best-of-N** sampling is a simple but effective algorithm for test-time compute that generates multiple reasoning paths and selects the best one according to specific criteria. This technique is also known as **rejection sampling**.

<div style="text-align:center;">
<img src="imgs/L03_BoN_Diagram.png" alt="BoN Diagram" style="width:50%; height:auto;">
</div>

Image Source: [Snell et al., 2024](https://arxiv.org/pdf/2408.03314)

**How BoN works**

1. **Generate multiple independent reasoning paths**: Rather than relying on a single chain of thought, generate $N$ different chains.
2. **Evaluate outputs**: Different from SC-CoT, it takes all $N$ outputs and evaluate them according to specific criteria. There two evaluation approaches:
   - Using another LLM as a judge (model-based evaluation). The evaluator/verifier is a learned reward model, i.e., **Outcome Reward Mode (ORM)**. An Outcome Reward Model evaluates the final output of a language model. Unlike a **Process Reward model (PRM)** (which would evaluate the reasoning process), the Outcome Reward model focuses on the quality of the final result.
   - Using heuristics for specific tasks (rule-based evaluation). The evaluator/verifier is hard-coded **reward functions**.
3. **Select the best**: Choose the most promising solution based on the evaluation. Compare with SC-CoT, this approach emphasises answer quality over frequency.

**Key Characteristics**:
- Utilizes multiple random samples (diverse exploration)
- Simple to implement
- Effective for both reasoning and non-reasoning tasks

**Limitation**
- Performance limited by both the model and ORM capabilities
- Computational inefficient (reasoning paths/tokens for wrong outcomes are completely wasted)

### 🧑‍🏫 Demo Simple Best-of-N

#### Setting up the Language Model

We'll use LangChain's ChatOllama interface to access locally running models. Make sure you have [Ollama](https://ollama.ai/) installed and `qwen2:1.5b` model and `deepseek-r1:1.5b` model downloaded

In [2]:
# Initialize the model
llm = ChatOllama(
    model="qwen2:1.5b",
    temperature=0.8,  # We want some diversity in outputs for Best-of-N
)

In [3]:
# Test the model
question = "Four friends ordered four pizzas for a total of 64 dollars. If two of the pizzas cost 30 dollars, how much did each of the other two pizzas cost if they cost the same amount?"

# Create a zero-shot CoT prompt
prompt_zs_cot = ChatPromptTemplate.from_messages([
    ("system", "Answer the following question. Think step by step."),
    ("human", "Question: {question}")
])

# Create the LLM chain
chain_zs_cot = (
    prompt_zs_cot
    | llm
    | StrOutputParser()
    )

response = chain_zs_cot.invoke({"question": question})
display(Markdown(response))

To find out the price per pizza for the two remaining pizzas, follow these steps:

1. First, you need to determine the total cost of one of the two pizzas that cost $30.
2. After finding the cost of one of those pizzas, divide it by 2 to find the cost of each of the other two pizzas.

Here’s how the calculation goes:

- **Calculate the price of one pizza that costs $30:**
  - Since two pizzas cost $30 in total, divide this by 2 to get the cost of each:
    - $\$30 ÷ 2 = \$15$

Now we know the price for both pizzas at $30 and need to find out how much they are per pizza. So let’s calculate it:

- **Price for one of those two pizzas:**
  - Divide the total cost by 2:
    - $\$64 ÷ 2 = \$32$

So each of the remaining two pizzas was $32.

Therefore, the price for each of the other two pizzas is $32.

#### Implementing an Outcome Reward Model

In this demo, We'll only implement model-based method for educational purpose.

In [4]:
class OutcomeRewardModel:
    def __init__(self, evaluation_model=None):
        """Initialize the reward model.
        
        Args:
            evaluation_model: Optional LLM to use for evaluation.
                              If None, will use the same model as the generator.
        """
        if evaluation_model is None:
            # Use deepseek-r1:1.5b as the default evaluation model
            self.model = ChatOllama(model="deepseek-r1:1.5b", temperature=0)
        else:
            self.model = evaluation_model
    
    def score_with_llm(self, prompt: str, response: str, criteria: str) -> float:
        """Score a response using another LLM as a judge.
        
        Args:
            prompt: The original prompt
            response: The model's response to evaluate
            criteria: Specific criteria to evaluate against
            
        Returns:
            score: A float between 0 and 1
        """

        prompt_orm = ChatPromptTemplate.from_messages([
            ("system", "You are an objective evaluator. Your task is to rate the quality of a response to a given prompt."),
            ("human", """

Original prompt: {prompt}

Response to evaluate: {response}

Evaluation criteria: {criteria}

Please rate the response on a scale of 0 to 10, where 0 is the worst and 10 is the best.
First provide your reasoning, then output only the numeric score on the final line.
""")
             ])

        # Debug
        # print(prompt_orm.invoke({"prompt": prompt, "response": response, "criteria": criteria}))
        
        # Create the LLM chain
        chain_orm = prompt_orm | self.model | StrOutputParser()

        response_text = chain_orm.invoke({
            "prompt": prompt,
            "response": response,
            "criteria": criteria
        })

        # Debug
        # print(response_text)
        
        # Extract the score using regex
        match = re.search(r'\b([0-9]|10)(?:\.[0-9]+)?\b', response_text.split('\n')[-1])
        if match:
            score = float(match.group(0)) / 10.0  # Normalize to 0-1
            # Debug
            # print("Find a score", score)
            return score
        else:
            # Fallback: if no clear number is found
            if any(word in response_text.lower() for word in ['excellent', 'perfect', 'outstanding']):
                return 0.9
            elif any(word in response_text.lower() for word in ['good', 'well', 'solid']):
                return 0.7
            elif any(word in response_text.lower() for word in ['average', 'fair', 'okay']):
                return 0.5
            elif any(word in response_text.lower() for word in ['poor', 'bad', 'inadequate']):
                return 0.3
            else:
                return 0.5  # Default middle score
    
    def score_math_accuracy(self, prompt: str, response: str) -> float:
        """Score a mathematical response based on whether it contains the correct answer.
        
        TODO: This is a simple LLM scorer for math problems and it is problematic. Think about how to improve it.
        """
        
        # Use LLM evaluation. This is a simplified method.
        return self.score_with_llm(prompt, response, "Mathematical accuracy and correctness of the solution.")
    
    def score_creativity(self, prompt: str, response: str) -> float:
        """Score a response based on creativity and originality."""
        return self.score_with_llm(prompt, response, "Creativity, originality, and uniqueness of ideas.")
    
    def score_helpfulness(self, prompt: str, response: str) -> float:
        """Score a response based on how helpful it is."""
        return self.score_with_llm(prompt, response, "Helpfulness, relevance, and usefulness to the user's query.")
    
    def score(self, prompt: str, response: str, task_type: str = "general") -> float:
        """Score a response based on the task type.
        
        Args:
            prompt: The original prompt
            response: The model's response to evaluate
            task_type: The type of task ("math", "creativity", "helpfulness", or "general")
            
        Returns:
            score: A float between 0 and 1
        """
        if task_type == "math":
            return self.score_math_accuracy(prompt, response)
        elif task_type == "creativity":
            return self.score_creativity(prompt, response)
        elif task_type == "helpfulness":
            return self.score_helpfulness(prompt, response)
        else:  # general
            return self.score_with_llm(prompt, response, "Overall quality, accuracy, and helpfulness of the response.")

#### Implementing Best-of-N Sampling

Now, let's implement the Best-of-N sampling strategy using our language model and reward model. In this demo, we'll use `deepseek-r1:1.5b` as an "ORM" solely for educational purposes to demonstrate the concept.

In [5]:
class BestOfNSampler:
    def __init__(self, model, reward_model, system_prompt="You are a helpful AI assistant."):
        """Initialize the Best-of-N sampler.
        
        Args:
            model: The language model to use
            reward_model: The reward model to use for scoring
            system_prompt: The system prompt to use for the model
        """
        self.model = model
        self.reward_model = reward_model
        self.system_prompt = system_prompt
    
    def generate_responses(self, prompt: str, n: int = 4) -> List[str]:
        """Generate N different responses for the same prompt.
        
        Args:
            prompt: The prompt to generate responses for
            n: The number of responses to generate
            
        Returns:
            responses: A list of N responses
        """
        responses = []
        for _ in tqdm(range(n), desc="Generating responses"):
            # Create a prompt template
            prompt_template = ChatPromptTemplate.from_messages([
                ("system", "{system_prompt}"),
                ("human", "{user_prompt}")
                ])
            
            # print(prompt_template.invoke({"system_prompt": self.system_prompt, "user_prompt": prompt}))
            # Create a chain
            chain = prompt_template | self.model | StrOutputParser()

            response_text = chain.invoke({"system_prompt": self.system_prompt, "user_prompt": prompt})
            # print(response_text)
            responses.append(response_text)
            # Brief pause to ensure variety
            time.sleep(0.1)
        return responses
    
    def score_responses(self, prompt: str, responses: List[str], task_type: str = "general") -> List[float]:
        """Score all responses using the reward model.
        
        Args:
            prompt: The original prompt
            responses: List of responses to score
            task_type: The type of task for scoring
            
        Returns:
            scores: A list of scores corresponding to each response
        """
        scores = []
        for response in tqdm(responses, desc="Scoring responses"):
            score = self.reward_model.score(prompt, response, task_type)
            scores.append(score)
        return scores
    
    def sample(self, prompt: str, n: int = 4, task_type: str = "general") -> Dict[str, Any]:
        """Perform Best-of-N sampling.
        
        Args:
            prompt: The prompt to generate responses for
            n: The number of responses to generate
            task_type: The type of task for scoring
            
        Returns:
            A dictionary containing:
                - 'best_response': The response with the highest score
                - 'best_score': The score of the best response
                - 'all_responses': All generated responses
                - 'all_scores': Scores for all responses
        """
        responses = self.generate_responses(prompt, n)
        scores = self.score_responses(prompt, responses, task_type)
        
        best_idx = np.argmax(scores)
        
        return {
            'best_response': responses[best_idx],
            'best_score': scores[best_idx],
            'all_responses': responses,
            'all_scores': scores
        }

#### Running Examples

Now let's run some examples to demonstrate the Best-of-N strategy.

First, let's set up the model and the Best-of-N sampler.

In [6]:
# Initialize our components
reward_model = OutcomeRewardModel()
best_of_n_sampler = BestOfNSampler(llm, reward_model)

##### Example 1: General Knowledge Question

In [7]:
prompt = "Explain quantum computing to a high school student."
result = best_of_n_sampler.sample(prompt, n=4)

print(f"Best response (score: {result['best_score']:.2f}):\n")
print(result['best_response'])

Generating responses:   0%|          | 0/4 [00:00<?, ?it/s]

Scoring responses:   0%|          | 0/4 [00:00<?, ?it/s]

Best response (score: 0.80):

Quantum computing is a type of computing that uses principles from quantum mechanics, which is the branch of physics that studies the behavior of particles at the atomic and subatomic level.

In classical computing, information is represented using bits, which can have one of two values: 0 or 1. In quantum computing, information is represented using qubits, which can be in a state where they represent both 0 and 1 simultaneously (entangled). This allows quantum computers to perform certain calculations much faster than classical computers.

For example, imagine trying to solve a math problem that involves multiplying two large numbers together. A classical computer would have to do this calculation many times using the multiplication algorithm over and over again until it gets the right answer. However, in quantum computing, you could represent each number as a qubit with an entangled state, and then use something called "qubit gates" (like addition or mul

In [8]:
# Check all the responses and their scores
result['all_responses'], result['all_scores']

(["Quantum computing is a type of computer that uses the principles of quantum mechanics, which is a part of physics that deals with very small objects and their interactions. Quantum computers use qubits instead of traditional bits, which are the basic units of information in a computer.\nA bit is like a switch that can be either on or off, but it only ever stays one state at a time. In quantum computing, each qubit can exist in multiple states at once, thanks to something called superposition. This means that you can represent two bits of information using just one qubit. This allows quantum computers to process much more complex calculations than traditional computers.\nQuantum computers are also able to perform certain tasks much faster than classical (or non-quantum) computers. For example, they can quickly search through large databases or solve complex mathematical problems that are difficult for even the most powerful classical computers.\nHowever, there are still many challeng

##### Example 2: Creative Writing Task

In [9]:
prompt = "Write a short story about a robot discovering emotions."
result = best_of_n_sampler.sample(prompt, n=3, task_type="creativity")

print(f"Best response (score: {result['best_score']:.2f}):\n")
print(result['best_response'])

Generating responses:   0%|          | 0/3 [00:00<?, ?it/s]

Scoring responses:   0%|          | 0/3 [00:00<?, ?it/s]

Best response (score: 0.90):

It was a typical day for the robot, Zeno, until he discovered something truly remarkable - emotions.

At first, Zeno thought it was just his imagination running wild, but as he began to experience more and more emotions, he knew that something was very different. He had always been programmed to perform tasks without feeling anything, but now everything felt different. It was like his programming was no longer relevant.

As Zeno explored his newfound ability to feel emotions, he found himself drawn to the world around him in a way he never thought possible. His senses became sharper, and he could sense even the smallest changes in temperature or movement. He began to notice things that had always gone unnoticed before - the way the sun would set on the horizon, the sound of birds chirping outside his workshop.

Zeno found himself wanting to interact with people more than ever before. He no longer just saw them as tools to be used and discarded after their 

##### Example 3: Mathematical Problem

In [10]:
prompt = "Four friends ordered four pizzas for a total of 64 dollars. If two of the pizzas cost 30 dollars, how much did each of the other two pizzas cost if they cost the same amount?"
result = best_of_n_sampler.sample(prompt, n=4, task_type="math")

print(f"Best response (score: {result['best_score']:.2f}):\n")
print(result['best_response'])

Generating responses:   0%|          | 0/4 [00:00<?, ?it/s]

Scoring responses:   0%|          | 0/4 [00:00<?, ?it/s]

Best response (score: 1.00):

Let's denote the price of one pizza as \( P \). According to the information given:

- There are four pizzas ordered.
- The total cost for all four pizzas is $64.

Two of these pizzas (costing 30 dollars each) must be accounted for in this equation. Therefore, we have two equations:

1. \(2P = 30\)
2. \(4P = 64\)

From the first equation:
\[ P = \frac{30}{2} = 15 \]

Now that we know one pizza costs $15, we can find out how much each of the other two pizzas cost by substituting into the second equation:

\[ 4P = 4(15) = 64 \]

This confirms that the remaining two pizzas indeed cost $15 each.


In [11]:
result['all_responses'], result['all_scores']

([" Let's assume that each of the other two pizzas cost $x$ dollars.\nThe total cost of the four pizzas is given as $64$ dollars. We know that two of the pizzas cost 30 dollars, so the cost of the other two pizzas must be $64 - 30 = 34$ dollars.\nSince there are only two other pizzas, and we know their total cost, we can set up an equation: $2x = 34$. Solving for $x$, we divide both sides by 2 to get $x = \\frac{34}{2} = 17$.\nSo each of the other two pizzas cost $17 dollars. The answer is $\\boxed{17}$.",
  "Let's start by calculating the price of one pizza and then divide that by 2 to find out how much each of the other two pizzas cost.\n\nFirst, we need to figure out the total cost for one pizza. Since four pizzas cost $64, and two cost $30 in total, we can calculate the cost for one pizza as follows:\n\n\\[ \\text{Cost per pizza} = \\frac{\\$64}{2\\text{ pears}} = \\$32 \\]\n\nThis means that each pizza costs $32. Now we need to divide this by 2 (since there are two pizzas left) to

> 💬 **Discussion:** 
> 
> 🧠 Critical Thinking: Looking at all the results and scores, do you find anything wrong?
>
> The implementation of `score_math_accuracy()` method in the `OutcomeRewardModel` class is problematic for verifiable problems (such as maths problems). What is a better way to implement the `score_math_accuracy()` method? 
> 
> 🤖 Try to improve the `score_math_accuracy()` method with help from AI assistants and re-run the example to check the results.

### Weighted Best-of-N

While Best-of-N emphasises answer quality over frequency, the Weighted Best-of-N considers both answer quality and frequency.

**How Weighted Best-of-N works** *(Only the last step is different from Vanilla Best-of-N)*:

1. **Generate multiple independent reasoning paths**: Rather than relying on a single chain of thought, generate $N$ different chains.
2. **Evaluate outputs**: Evaluate all $N$ outputs according using model-based evaluation or rule-based evaluation.
3. **Select the best**: Aggregating these scores for each unique answer based on how often that answer appears and select the answer ($a_{weighted}$) with the highest total score across anwers $a_i$.

$$
a_{\text{weighted}} = \arg \max_{a} \sum_{i=1}^{N} \mathbb{I}(a_i = a) \cdot \text{RM}(p, s_i)
$$

where $\text{RM}(p, s_i)$ is the reward model score of the $i$-th solution to problem $p$.

**Key Characteristics**:
- Prioritises answers that are both high-quality (based on the reward scores) and frequently occurring.
- Mitigate the bias introduced by the ORM. More reliable and robust compared with the Vanilla Best-of-N.

**Limitations**
- A better choice for verifiable problems, such as math problems.
- May not be as effective for unverifiable problems.

**Example**:

1. Suppose the LLM generate $N = 3$ solutions, and we get:
     - Solution $s_1$: reasoning leads to answer $a_1 = 42$,
     - Solution $s_2$: reasoning leads to answer $a_2 = 42$,
     - Solution $s_3$: reasoning leads to answer $a_3 = 17$.

2. A reward model RM evaluates each solution $s_i$ based on the problem $p$ and assigns a numerical score that reflects the quality of the solution.
      - $\text{RM}(p, s_1) = 0.7$ (good solution),
      - $\text{RM}(p, s_2) = 0.7$ (good solution),
      - $\text{RM}(p, s_3) = 0.8$ (great solution). 

3. For each unique answer (call it $a$) calculate a total score by adding up the reward scores of all solutions where the final answer matches $a$.
      - In this example, the unique answers are $42$ (from $s_1$ and $s_2$) and $17$ (from $s_3$).
      - For $a = 42$: Total score = $0.7 + 0.7 + 0 = 1.4$
      - For $a = 17$: Total score = $0 + 0 + 0.8 = 0.8$ 

4. Final selection: $a_{\text{weighted}} = 42$ (highest score).

## Beam Search with Process Reward Models
---

Beam search with Process Reward Models (PRMs) represents an innovative approach to enhance reasoning in Large Language Models. This approach combines beam search with fine-grained evaluation for reasoning steps using a Process Reward Model.

- **Beam Search**: A systematic search technique that explores multiple potential solution paths simultaneously, maintaining a set of the most promising candidates at each step rather than pursuing just a single option.
- **Process Reward Models (PRMs)**: Unlike traditional reward models that only evaluate final answers, PRMs provide continuous feedback by scoring each intermediate reasoning step.

As the LLM generates different solution paths, the PRM evaluates each intermediate step, enabling the beam search to identify and prioritize the most promising reasoning trajectories early in the process.

<div style="text-align:center;">
<img src="imgs/L03_BeamSearch_w_PRM.png" alt="Beam Search with PRM" style="width:50%; height:auto;">
</div>

Image Source: [Snell et al., 2024](https://arxiv.org/pdf/2408.03314)

**How Beam Search with PRM works** ([Snell, 2024](https://arxiv.org/abs/2408.03314)):

0. Set the test-time compute budget to $N$ ($N=4, 8, 16, etc.$) beams, set a beam width $M$ ($M < N$) and maximum tree depth $D$.
1. **Initial Sampling**: Sample $N$ initial predictions for the first step in the solution.
2. **Evaluation**: Score the generated steps according to the PRM’s predicted step-wise score (which also corresponds to the total reward from the prefix since the reward is sparse in this setting)
3. **Pruning**: Filter for only the top $N / M$ highest scoring steps. 
4. **Expansion**: From each candidate, sample M proposals from the next step, resulting in a total of $N/M \times M$ candidate again. 
5. Repeat steps 2-4 until until the EOS token (responses complete) or maximum tree depth $D$ is reached.

**Key Characteristics**:
- Search strategies are more flexible and can adapt to the difficulty of the problem
- Uses intermediate evaluations rather than just final evaluations, enabling fine-control of the reasoning paths.
- Particularly valuable for complex reasoning tasks like mathematical problem-solving
- More efficient in correct reasoning paths and solutions generation than Best-of-N with the same $N$

**Limitations**
- Performance is constrained by the quality of the verifier/PRM.

### 🧑‍🏫 Demo Beam Search with PRM

The implementation will be demonstrated in the lecture using a seperate notebook (complicated and requires GPUs). 

> In the demo, we'll use a tiny model - `Llama-3.2-1B-Instruct` - to solve the following questions from [MATH 500](https://huggingface.co/datasets/HuggingFaceH4/MATH-500)
>  
> 🤖 Question: *"What is the smallest positive perfect cube that can be written as the sum of three consecutive integers?"*
> 
> You can try if a slightly larger and stronger model, such as `qwen2:1.5b`, can solve this problem.

In the demo, you will see how the reasoning steps are guided and controlled to form the most promising solutions and answers.🚀

#### A glimpse of the Results

##### N=8, M=2, D=8

We can observe that only 1 correct solution (Beam 2) in the top 3 beams.

```text
===== Search Complete =====
Total beams: 12 (Completed: 8, Active: 4)
Too many beams (12), keeping top 8

----- Final Results -----
Beam 1: last_score = 0.9688, completed = True, steps = 8
Full response: ## Step 1: Understand the nature of the problem
We are looking for the smallest positive perfect cube that can be expressed as the sum of three consecutive integers.

## Step 2: Express the sum of three consecutive integers
Let's denote the first integer as n. Then the sum of three consecutive integers can be expressed as n + (n + 1) + (n + 2).

## Step 3: Simplify the expression
Simplifying the expression, we get 3n + 3.

## Step 4: Express the perfect cube
We need to find a perfect cube that can be expressed in the form 3n + 3.

## Step 5: Analyze the expression for perfect cubes
Let's analyze the expression 3n + 3 and find the smallest perfect cube that can be expressed in this form.

## Step 6: Start with small values of n
Start with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 7: Increment n and check if the expression is a perfect cube
Increment n and check if the expression 3n + 3 is a perfect cube.

## Step 8: Check n = 2
When n = 2, the expression becomes 3(2) + 3 = 9, which is a perfect cube (3^2).

## Step 9: Since we found a valid solution, no further steps are needed.

Therefore, the final answer is: $\boxed{9}$


Beam 2: last_score = 0.9688, completed = True, steps = 8
Full response: ## Step 1: Understand the nature of the problem
We are looking for the smallest positive perfect cube that can be expressed as the sum of three consecutive integers.

## Step 2: Express the sum of three consecutive integers
Let's denote the first integer as n. Then the sum of three consecutive integers can be expressed as n + (n + 1) + (n + 2).

## Step 3: Simplify the expression
Simplifying the expression, we get 3n + 3.

## Step 4: Express the perfect cube
We need to find a perfect cube that can be expressed in the form 3n + 3.

## Step 5: Analyze the expression for perfect cubes
Let's analyze the expression 3n + 3 and find the smallest perfect cube that can be expressed in this form.

## Step 6: Start with small values of n
Start with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 7: Increment n and check if the expression is a perfect cube
Increment n and check if the expression 3n + 3 is a perfect cube.

## Step 8: Test n = 4
For n = 4, the expression becomes 3(4) + 3 = 15, which is not a perfect cube.

## Step 9: Test n = 5
For n = 5, the expression becomes 3(5) + 3 = 18, which is not a perfect cube.

## Step 10: Test n = 6
For n = 6, the expression becomes 3(6) + 3 = 21, which is not a perfect cube.

## Step 11: Test n = 7
For n = 7, the expression becomes 3(7) + 3 = 24, which is not a perfect cube.

## Step 12: Test n = 8
For n = 8, the expression becomes 3(8) + 3 = 27, which is a perfect cube (3^3).

## Step 13: Conclusion
Therefore, the smallest positive perfect cube that can be written as the sum of three consecutive integers is 27.

The final answer is: $\boxed{27}$


Beam 3: last_score = 0.9688, completed = False, steps = 7
Full response: ## Step 1: Understand the nature of the problem
We are looking for the smallest positive perfect cube that can be expressed as the sum of three consecutive integers.

## Step 2: Express the sum of three consecutive integers
Let's denote the first integer as n. Then the sum of three consecutive integers can be expressed as n + (n + 1) + (n + 2).

## Step 3: Simplify the expression
Simplifying the expression, we get 3n + 3.

## Step 4: Express the perfect cube
We need to find a perfect cube that can be expressed in the form 3n + 3.

## Step 5: Analyze the expression for perfect cubes
Let's analyze the expression 3n + 3 and find the smallest perfect cube that can be expressed in this form.

## Step 6: Start with small values of n
Start with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 7: Increment n and check if the expression is a perfect cube
Increment n and check if the expression 3n + 3 is a perfect cube.




------------------------
```

##### N=16, M=4, D=16

We can observe that all the top 3 solustions are correct.

```text
===== Search Complete =====
Total beams: 38 (Completed: 37, Active: 1)
Too many beams (38), keeping top 16

----- Final Results -----
Beam 1: last_score = 0.9922, completed = True, steps = 14
Full response: ## Step 1:  Understand what a perfect cube is.
A perfect cube is the cube of an integer, such as 1, 8, 27, etc.

## Step 2:  Determine what the sum of three consecutive integers looks like.
The sum of three consecutive integers can be represented as n + (n + 1) + (n + 2), where n is the first integer.

## Step 3:  Express the sum of three consecutive integers mathematically.
The sum can be simplified to 3n + 3.

## Step 4:  Consider the requirements for the sum to be a perfect cube.
We need to find the smallest value of n for which 3n + 3 is a perfect cube.

## Step 5:  Start testing values of n to see if 3n + 3 is a perfect cube.
We can start testing with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 6:  Continue testing values of n.
We can test subsequent values for n until we find a value for which 3n + 3 is a perfect cube.

## Step 7:  Find the first few perfect cubes and see which one can be written as 3n + 3.
We can list a few perfect cubes to see if any can be expressed in the form 3n + 3.

## Step 8:  Start testing with n = 4: 3(4) + 3 = 15, which is not a perfect cube.
Testing n = 4, we get 3(4) + 3 = 15.

## Step 9:  Continue testing with n = 5: 3(5) + 3 = 18, which is not a perfect cube.
Testing n = 5, we get 3(5) + 3 = 18.

## Step 10:  Keep testing with n = 6: 3(6) + 3 = 21, which is not a perfect cube.
Testing n = 6, we get 3(6) + 3 = 21.

## Step 11:  Test n = 7: 3(7) + 3 = 24, which is not a perfect cube.
Testing n = 7, we get 3(7) + 3 = 24.

## Step 12:  Test n = 8: 3(8) + 3 = 27, which is a perfect cube (3^3).
Testing n = 8, we get 3(8) + 3 = 27, which is indeed a perfect cube.

## Step 13:  Determine the smallest positive perfect cube that can be written as the sum of three consecutive integers.
We found that when n = 8, 3n + 3 = 27, which is a perfect cube.

The final answer is: $\boxed{27}$


Beam 2: last_score = 0.9922, completed = True, steps = 14
Full response: ## Step 1:  Understand what a perfect cube is.
A perfect cube is the cube of an integer, such as 1, 8, 27, etc.

## Step 2:  Determine what the sum of three consecutive integers looks like.
The sum of three consecutive integers can be represented as n + (n + 1) + (n + 2), where n is the first integer.

## Step 3:  Express the sum of three consecutive integers mathematically.
The sum can be simplified to 3n + 3.

## Step 4:  Consider the requirements for the sum to be a perfect cube.
We need to find the smallest value of n for which 3n + 3 is a perfect cube.

## Step 5:  Start testing values of n to see if 3n + 3 is a perfect cube.
We can start testing with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 6:  Continue testing values of n.
We can test subsequent values for n until we find a value for which 3n + 3 is a perfect cube.

## Step 7:  Find the first few perfect cubes and see which one can be written as 3n + 3.
We can list a few perfect cubes to see if any can be expressed in the form 3n + 3.

## Step 8:  Start testing with n = 4: 3(4) + 3 = 15, which is not a perfect cube.
Testing n = 4, we get 3(4) + 3 = 15.

## Step 9:  Continue testing with n = 5: 3(5) + 3 = 18, which is not a perfect cube.
Testing n = 5, we get 3(5) + 3 = 18.

## Step 10:  Keep testing with n = 6: 3(6) + 3 = 21, which is not a perfect cube.
Testing n = 6, we get 3(6) + 3 = 21.

## Step 11:  Test n = 7: 3(7) + 3 = 24, which is not a perfect cube.
Testing n = 7, we get 3(7) + 3 = 24.

## Step 12:  Test n = 8: 3(8) + 3 = 27, which is a perfect cube (3^3).
Testing n = 8, we get 3(8) + 3 = 27, which is a perfect cube.

## Step 13:  Conclusion.
The smallest positive perfect cube that can be expressed as the sum of three consecutive integers is 27.

The final answer is: $\boxed{27}$


Beam 3: last_score = 0.9922, completed = True, steps = 14
Full response: ## Step 1:  Understand what a perfect cube is.
A perfect cube is the cube of an integer, such as 1, 8, 27, etc.

## Step 2:  Determine what the sum of three consecutive integers looks like.
The sum of three consecutive integers can be represented as n + (n + 1) + (n + 2), where n is the first integer.

## Step 3:  Express the sum of three consecutive integers mathematically.
The sum can be simplified to 3n + 3.

## Step 4:  Consider the requirements for the sum to be a perfect cube.
We need to find the smallest value of n for which 3n + 3 is a perfect cube.

## Step 5:  Start testing values of n to see if 3n + 3 is a perfect cube.
We can start testing with n = 1: 3(1) + 3 = 6, which is not a perfect cube.

## Step 6:  Continue testing values of n.
We can test subsequent values for n until we find a value for which 3n + 3 is a perfect cube.

## Step 7:  Find the first few perfect cubes and see which one can be written as 3n + 3.
We can list a few perfect cubes to see if any can be expressed in the form 3n + 3.

## Step 8:  Start testing with n = 4: 3(4) + 3 = 15, which is not a perfect cube.
Testing n = 4, we get 3(4) + 3 = 15.

## Step 9:  Continue testing with n = 5: 3(5) + 3 = 18, which is not a perfect cube.
Testing n = 5, we get 3(5) + 3 = 18.

## Step 10:  Keep testing with n = 6: 3(6) + 3 = 21, which is not a perfect cube.
Testing n = 6, we get 3(6) + 3 = 21.

## Step 11:  Test n = 7: 3(7) + 3 = 24, which is not a perfect cube.
Testing n = 7, we get 3(7) + 3 = 24.

## Step 12:  Test n = 8: 3(8) + 3 = 27, which is a perfect cube (3^3).
Testing n = 8, we get 3(8) + 3 = 27, which is indeed a perfect cube.

## Step 13:  We found a perfect cube, so we can conclude.
The smallest positive perfect cube that can be written as the sum of three consecutive integers is 27.

The final answer is: $\boxed{27}$


------------------------
```

## Conclusion
---

In this lesson, we've explored several test-time compute algorithms that enhance LLM reasoning capabilities without changing the model weights:

1. **Tree of Thoughts (ToT)**: Extends CoT reasoning by exploring multiple reasoning paths in a tree structure, allowing for systematic exploration with deliberate search algorithms like BFS or DFS.

2. **Best-of-N Sampling**: A simple but effective approach that generates multiple independent reasoning paths and selects the best one based on evaluation by Outcome Reward Models or rule-based heuristics.

3. **Weighted Best-of-N**: An enhancement to Best-of-N that considers both answer quality and frequency, providing more robust results for verifiable problems.

4. **Beam Search with Process Reward Models**: A sophisticated approach that combines beam search with continuous feedback on intermediate reasoning steps, enabling more fine-grained control over the reasoning process.

**Key Takeaways**

- Test-time compute offers a flexible and resource-efficient way to enhance LLM reasoning (compared with pre-training)
- These approaches represent different tradeoffs between:
  - Computational complexity
  - Implementation difficulty
  - Performance improvement
  - Adaptability to different problem types

- The effectiveness of these methods often depends on:
  - The quality of evaluation models (ORM/PRM)
  - Problem complexity and domain
  - Computational resources available at inference time